## Extract Text(Headers, Sub-Headers & Paragraph) from PDF Resumes Using PyMuPDF and Python

#### Reference
https://www.neurond.com/blog/extract-text-from-pdf-pymupdf-and-python

In [1]:
import fitz

In [13]:
! pip install pymupdf unidecode

                                              0.0/235.9 kB ? eta -:--:--
     ------                                41.0/235.9 kB 991.0 kB/s eta 0:00:01
     --------------                        92.2/235.9 kB 871.5 kB/s eta 0:00:01
     ---------------------                143.4/235.9 kB 944.1 kB/s eta 0:00:01
     --------------------------           174.1/235.9 kB 871.5 kB/s eta 0:00:01
     -----------------------------------  235.5/235.9 kB 962.7 kB/s eta 0:00:01
     ------------------------------------ 235.9/235.9 kB 848.2 kB/s eta 0:00:00


In [2]:
pdf_path = "C:/Users/achyuta.sahoo/Documents/work/NL_to_SQL/prod/nl_to_sql/data/pdf_docs/AMIND-Contingent Worker Agreement _ revised (1).pdf"
doc = fitz.open(pdf_path)
for page in doc:
    text = page.get_text()
    print(text)
    break

 
Page 1           Applied Materials Confidential 
2019-07-17 AMIND 
 
CONTINGENT WORKER CONFIDENTIALITY AGREEMENT 
 
This Contingent Worker Confidentiality Agreement (this “Agreement”) is made and entered into this ______ 
day of ____________________, 20__ by and between Applied Materials, Inc., and its subsidiaries and affiliates, 
including 
Applied 
Materials 
India 
Private 
Limited 
 
(collectively 
“Applied”), 
and 
__________________________________________________________(“Contingent Worker”), an employee of 
__________________________________________________________________________(“Service Provider”). 
Whereas, Applied and Service Provider have entered into an agreement whereby Service Provider places 
Service Provider’s employees or contractors at the site of Applied or an Applied customer to perform services for 
Applied;  
Now therefore, in consideration of Service Provider’s assignment of Contingent Worker to the premises of 
Applied or an Applied customer as required by

In [4]:
for page in doc:
    output = page.get_text("blocks")                   
    previous_block_id = 0 # Set a variable to mark the block id
    for block in output:
        if block[6] == 0: # We only take the text
            if previous_block_id != block[5]: # Compare the block number 
                print("\n")
            print(block[4])

 



Page 1           Applied Materials Confidential 
2019-07-17 AMIND 
 



CONTINGENT WORKER CONFIDENTIALITY AGREEMENT 



 
This Contingent Worker Confidentiality Agreement (this “Agreement”) is made and entered into this ______ 



day of ____________________, 20__ by and between Applied Materials, Inc., and its subsidiaries and affiliates, 
including 
Applied 
Materials 
India 
Private 
Limited 
 
(collectively 
“Applied”), 
and 



__________________________________________________________(“Contingent Worker”), an employee of 
__________________________________________________________________________(“Service Provider”). 



Whereas, Applied and Service Provider have entered into an agreement whereby Service Provider places 



Service Provider’s employees or contractors at the site of Applied or an Applied customer to perform services for 
Applied;  



Now therefore, in consideration of Service Provider’s assignment of Contingent Worker to the premises of 



Applied or an Appl

In [3]:
block_dict = {}
page_num = 1
for page in doc: # Iterate all pages in the document
      file_dict = page.get_text('dict') # Get the page dictionary 
      block = file_dict['blocks'] # Get the block information
      block_dict[page_num] = block # Store in block dictionary
      page_num += 1 # Increase the page value by 1
#block_dict

In [4]:
import re 
import numpy as np
import pandas as pd
from unidecode import unidecode 

spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])
rows = []
for page_num, blocks in block_dict.items():
    for block in blocks:
        if block['type'] == 0:
            for line in block['lines']:
                for span in line['spans']:
                    xmin, ymin, xmax, ymax = list(span['bbox'])
                    font_size = span['size']
                    text = unidecode(span['text'])
                    span_font = span['font']
                    is_upper = False
                    is_bold = False 
                    if "bold" in span_font.lower():
                        is_bold = True 
                    if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                        is_upper = True
                    if text.replace(" ","") !=  "":
                        rows.append((page_num,xmin, ymin, xmax, ymax, text, is_upper, is_bold, span_font, font_size))

span_df = pd.DataFrame(rows, columns=['page_num','xmin','ymin','xmax','ymax', 'text', 'is_upper','is_bold','span_font', 'font_size'])

In [5]:
span_df

,page_num,xmin,ymin,xmax,ymax,text,is_upper,is_bold,span_font,font_size
0,1,50.474998,721.858582,215.279999,733.597778,Page 1 Applied Materials Confidential,False,False,TimesNewRomanPSMT,8.8
1,1,50.474998,732.283569,128.770004,744.022766,2019-07-17 AMIND,True,False,TimesNewRomanPSMT,8.8
2,1,149.800003,48.357185,462.384796,63.869186,CONTINGENT WORKER CONFIDENTIALITY AGREEMENT,True,True,TimesNewRomanPS-BoldMT,11.2
3,1,86.500000,73.876381,342.876312,88.817184,This Contingent Worker Confidentiality Agreeme...,False,False,TimesNewRomanPSMT,11.2
4,1,342.850006,73.977180,394.582855,89.489182,Agreement,False,True,TimesNewRomanPS-BoldMT,11.2
...,...,...,...,...,...,...,...,...,...,...
443,9,36.049999,537.556396,579.549988,552.497192,"prepare an application to obtain a license, be...",False,False,TimesNewRomanPSMT,11.2
444,9,36.049999,550.376404,161.400009,565.317200,to be granted such license.,False,False,TimesNewRomanPSMT,11.2
445,9,36.049999,587.307190,64.206795,602.819214,*Note,False,True,TimesNewRomanPS-BoldMT,11.2
446,9,64.074997,587.206421,578.710999,602.147217,: Though Macau is recognized as a special admi...,False,False,TimesNewRomanPSMT,11.2


In [6]:
span_scores = []
span_num_occur = {}
special = '[(_:/,#%\=@)]'

for index, span_row in span_df.iterrows():
    score = round(span_row.font_size)
    text = span_row.text
    if not re.search(special, text):
        if span_row.is_bold:
            score +=1 
        if span_row.is_upper:
            score +=1
    span_scores.append(score)

values, counts = np.unique(span_scores, return_counts=True)

In [7]:
#values, counts = np.unique(span_scores, return_counts=True)
style_dict = {}
for value, count in zip(values, counts):
    style_dict[value] = count
sorted(style_dict.items(), key=lambda x: x[1])

[(8, 3), (10, 6), (9, 15), (13, 19), (12, 32), (11, 373)]

In [8]:
p_size = max(style_dict, key=style_dict.get)
idx = 0
tag = {}
for size in sorted(values, reverse = True):
    idx += 1
    if size == p_size:
        idx = 0
        tag[size] = 'p'
    if size > p_size:
        tag[size] = 'h{0}'.format(idx)
    if size < p_size:
        tag[size] = 's{0}'.format(idx)

In [9]:
span_tags = [tag[score] for score in span_scores]
span_df['tag'] = span_tags
span_df

,page_num,xmin,ymin,xmax,ymax,text,is_upper,is_bold,span_font,font_size,tag
0,1,50.474998,721.858582,215.279999,733.597778,Page 1 Applied Materials Confidential,False,False,TimesNewRomanPSMT,8.8,s2
1,1,50.474998,732.283569,128.770004,744.022766,2019-07-17 AMIND,True,False,TimesNewRomanPSMT,8.8,s1
2,1,149.800003,48.357185,462.384796,63.869186,CONTINGENT WORKER CONFIDENTIALITY AGREEMENT,True,True,TimesNewRomanPS-BoldMT,11.2,h1
3,1,86.500000,73.876381,342.876312,88.817184,This Contingent Worker Confidentiality Agreeme...,False,False,TimesNewRomanPSMT,11.2,p
4,1,342.850006,73.977180,394.582855,89.489182,Agreement,False,True,TimesNewRomanPS-BoldMT,11.2,h2
...,...,...,...,...,...,...,...,...,...,...,...
443,9,36.049999,537.556396,579.549988,552.497192,"prepare an application to obtain a license, be...",False,False,TimesNewRomanPSMT,11.2,p
444,9,36.049999,550.376404,161.400009,565.317200,to be granted such license.,False,False,TimesNewRomanPSMT,11.2,p
445,9,36.049999,587.307190,64.206795,602.819214,*Note,False,True,TimesNewRomanPS-BoldMT,11.2,h2
446,9,64.074997,587.206421,578.710999,602.147217,: Though Macau is recognized as a special admi...,False,False,TimesNewRomanPSMT,11.2,p


In [10]:
headings_list = []
text_list = []
page_list = []
tmp = []
heading = ''                                                                                                                
for index, span_row in span_df.iterrows():
    text = span_row.text
    tag = span_row.tag
    page_num = span_row.page_num
    page_list.append(page_num)
    if 'h' in tag:
        headings_list.append(text)
        text_list.append('\n'.join(tmp))
        tmp = []
        heading = text
    else:
        tmp.append(text)

text_list.append('\n'.join(tmp))
text_list = text_list[1:]
text_df = pd.DataFrame(zip(page_list,headings_list, text_list),columns=['page_num','heading', 'content'] )

In [11]:
text_df

,page_num,heading,content
0,1,CONTINGENT WORKER CONFIDENTIALITY AGREEMENT,This Contingent Worker Confidentiality Agreeme...
1,1,Agreement,""") is made and entered into this ______ \nday ..."
2,1,Applied,"""), \nand \n__________________________________..."
3,1,Contingent Worker,"""), an employee of \n_________________________..."
4,1,Service Provider,"""). \nWhereas, Applied and Service Provider ha..."
5,1,1.,
6,1,CONFIDENTIAL INFORMATION,"a.\nAs used in this section, ""Confidential Inf..."
7,1,2.,
8,1,CONFIDENTIAL INFORMATION TO COURTS OR GOVERNME...,The obligation of confidentiality on the Conti...
9,1,3.,


In [13]:
import re 
import numpy as np
import pandas as pd
from unidecode import unidecode 
import fitz

class PdfHeaderGenerator():
    def __init__(self) -> None:
        pass

    def read_pdf(self,pdf_path):
        self.doc = fitz.open(pdf_path)

    def get_block_dict(self,):
        self.block_dict = {}
        page_num = 1
        for page in self.doc: # Iterate all pages in the document
            file_dict = page.get_text('dict') # Get the page dictionary 
            block = file_dict['blocks'] # Get the block information
            self.block_dict[page_num] = block # Store in block dictionary
            page_num += 1 # Increase the page value by 1
    
    def get_span_df(self,):
        spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])
        rows = []
        for page_num, blocks in self.block_dict.items():
            for block in blocks:
                if block['type'] == 0:
                    for line in block['lines']:
                        for span in line['spans']:
                            xmin, ymin, xmax, ymax = list(span['bbox'])
                            font_size = span['size']
                            text = unidecode(span['text'])
                            span_font = span['font']
                            is_upper = False
                            is_bold = False 
                            if "bold" in span_font.lower():
                                is_bold = True 
                            if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                                is_upper = True
                            if text.replace(" ","") !=  "":
                                rows.append((page_num,xmin, ymin, xmax, ymax, text, is_upper, is_bold, span_font, font_size))

        self.span_df = pd.DataFrame(rows, columns=['page_num','xmin','ymin','xmax','ymax', 'text', 'is_upper','is_bold','span_font', 'font_size'])
    
    def get_span_tag(self,):
        span_scores = []
        span_num_occur = {}
        special = '[(_:/,#%\=@)]'

        for index, span_row in self.span_df.iterrows():
            score = round(span_row.font_size)
            text = span_row.text
            if not re.search(special, text):
                if span_row.is_bold:
                    score +=1 
                if span_row.is_upper:
                    score +=1
            span_scores.append(score)

        values, counts = np.unique(span_scores, return_counts=True)

        style_dict = {}
        for value, count in zip(values, counts):
            style_dict[value] = count
        #sorted(style_dict.items(), key=lambda x: x[1])

        p_size = max(style_dict, key=style_dict.get)
        idx = 0
        tag = {}
        for size in sorted(values, reverse = True):
            idx += 1
            if size == p_size:
                idx = 0
                tag[size] = 'p'
            if size > p_size:
                tag[size] = 'h{0}'.format(idx)
            if size < p_size:
                tag[size] = 's{0}'.format(idx)
        
        span_tags = [tag[score] for score in span_scores]
        self.span_df['tag'] = span_tags
        #span_df
    
    def get_heading_content(self,):
        headings_list = []
        text_list = []
        page_list = []
        tmp = []
        heading = ''                                                                                                                
        for index, span_row in self.span_df.iterrows():
            text = span_row.text
            tag = span_row.tag
            page_num = span_row.page_num
            header_not_found = True
            if 'h' in tag:
                headings_list.append(text)
                text_list.append('\n'.join(tmp))
                page_list.append(page_num)
                tmp = []
                heading = text
                header_not_found = False
            else:
                tmp.append(text)
        
        text_list.append('\n'.join(tmp))
        text_list = text_list[1:]
        text_df = pd.DataFrame(zip(page_list,headings_list, text_list),columns=['page_num','heading', 'content'])

        return text_df
    
    def text_cleansing(self,df):
        df['heading'] = df['heading'].apply(lambda x: x.replace("\n"," ").lower())
        df['content'] = df['content'].apply(lambda x: x.replace("\n"," ").lower())

        return df

    
    def get_headers_with_content(self,path,max_seq_length=150):
        return_dict = dict()
        try:
            self.read_pdf(path)
            self.get_block_dict()
            self.get_span_df()
            self.get_span_tag()
            df = self.get_heading_content()
            df = self.text_cleansing(df)
            
            return df

            for each_page in df['page_num'].unique():
                return_dict[each_page] = {}
                headings = df[df['page_num']==each_page]['heading'].to_list()
                headings = [i for i in headings if len(i.strip())>3]
                return_dict[each_page]['headers'] = headings
                return_dict[each_page]['content'] = df[df['page_num']==each_page]['content'].to_list()
        except Exception as err:
            print(err)


        return return_dict



In [25]:
#if __name__ == '__main__':
pdf_path = "C:/Users/achyuta.sahoo/Documents/work/NL_to_SQL/prod/nl_to_sql/data/pdf_docs/AMIND-Contingent Worker Agreement _ revised (1).pdf"
header_gen = PdfHeaderGenerator()
#data = header_gen.get_headers_with_content(pdf_path)
#print(data)
header_gen.read_pdf(pdf_path)
for page_no, page in enumerate(header_gen.doc):
    data = page.get_text()
    print(f"page number: {page_no}")
    print(data)
    print("\n\n")

page number: 0
 
Page 1           Applied Materials Confidential 
2019-07-17 AMIND 
 
CONTINGENT WORKER CONFIDENTIALITY AGREEMENT 
 
This Contingent Worker Confidentiality Agreement (this “Agreement”) is made and entered into this ______ 
day of ____________________, 20__ by and between Applied Materials, Inc., and its subsidiaries and affiliates, 
including 
Applied 
Materials 
India 
Private 
Limited 
 
(collectively 
“Applied”), 
and 
__________________________________________________________(“Contingent Worker”), an employee of 
__________________________________________________________________________(“Service Provider”). 
Whereas, Applied and Service Provider have entered into an agreement whereby Service Provider places 
Service Provider’s employees or contractors at the site of Applied or an Applied customer to perform services for 
Applied;  
Now therefore, in consideration of Service Provider’s assignment of Contingent Worker to the premises of 
Applied or an Applied customer

In [34]:
from nltk.tokenize import sent_tokenize, word_tokenize

train_sentences = []
for page_no, page in enumerate(header_gen.doc):
    data = page.get_text()
    print(f"page number: {page_no}")
    #print(data)
    #print("\n\n")
    sentences = sent_tokenize(data)
    sentences = [i.strip().replace('\n','').replace('_','').lower() for i in sentences]
    train_sentences.extend(sentences)
    #print(sentences)
    #break
print(f"total number of sentences: {len(train_sentences)}")

page number: 0
page number: 1
page number: 2
page number: 3
page number: 4
page number: 5
page number: 6
page number: 7
page number: 8
total number of sentences: 130


In [35]:
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from torch.utils.data import DataLoader

# Define your sentence transformer model using CLS pooling
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# Define a list with sentences (1k - 100k sentences)
#train_sentences = ["Your set of sentences",
#                   "Model will automatically add the noise", 
#                   "And re-construct it",
#                   "You should provide at least 1k sentences"]

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 3e-5},
    show_progress_bar=True
)

model.save('models/bert-base-uncased-local')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
When tie_encoder_decoder=True, the decoder_name_or_path will be invalid.
Some weights of the model checkpoint at bert-base-uncased were not used when initi

The following encoder weights were not tied to the decoder ['bert/pooler']


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17 [00:00<?, ?it/s]

In [36]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim


model_local = SentenceTransformer(r"models/bert-base-uncased-local")

In [39]:
embd_a = model_local.encode("give me former employer details from WORKER CONFIDENTIALITY AGREEMENT ?")
#embd_b = model_local.encode("""-shore, and far-shore.   machine:   we are taking advantage of best-in-class technology to maximize our productivity, speed, and quality.  this is driving our innovation, and our clients’ – especially in the way business-transforming insights are  generated, used, and shared.   by combining mind+machine, we are able to power client success – and our own   3.3  our values   always rigorous.   always refreshing.   always responsive       always rigorous   uncompromising quality   unrivalled expertise   ultra  efficient   what does it  mean for our  clients?   what does  this mean for  us?   evalueserve consistently  delivers excellence   we deliver excellence by  adhering to the highest  professional and ethical  standards   evalueserve's  promise to  you   we will set and follow the  highest professional and  ethical standards to create  an environment where  excellence is applauded   we can rely on evalueserve  to create best practice  mind+machine solutions for  us, every time   we combine our expertise  and  our unique technologies  to create best practice  solutions that we are proud  of   we will continually develop  your expertise and expose  you to our unique  technologies to create best- practice solutions for  evalueserve's clients   evalueserve always finds  the shortest route to the best  result   we will evaluate all possible  options while finding the  shortest route to the best  result   we will invest in the best  methods, tools, and  technologies – to automate,  streamline, and enhance the  way you""")
embd_b = model_local.encode("""he Contingent Worker shall return these items to Applied upon termination of his/her work assignment or at any time upon request by Applied. 6. INFORMATION OF FORMER EMPLOYER Contingent Worker shall not disclose or use for Applied’s benefit any confidential or proprietary information  of Contingent Worker’s former employer(s) or any other third parties to which Contingent Worker has a  confidentiality obligation. The Contingent Worker must not bring onto the premises of Applied any nonpublic documents or any other property belonging to any former employer(s) or any other third parties unless  permitted to do so in advance in writing by such party.""")

sim_score = cos_sim(embd_a, embd_b)

print(sim_score)

tensor([[0.8710]])


In [61]:
text = "give me former employer details from WORKER CONFIDENTIALITY AGREEMENT ?"
print(f"length of doc: {len(text)}")

def get_index_slicer_range(size=0,window=100,overlap=0):
    prev = 0
    output = []
    for j in [i for i in range(window,size+window,window)]:
        output.append((prev,j))
        prev = j-overlap
    
    return output

def get_sliced_text(text):
    index_slicer = get_index_slicer_range(len(text),10,5)
    print(index_slicer)
    sliced_text = [text[i:j] for i,j in index_slicer]
    return sliced_text

print(get_sliced_text(text))

length of doc: 71
[(0, 10), (5, 20), (15, 30), (25, 40), (35, 50), (45, 60), (55, 70), (65, 80)]
['give me fo', 'me former emplo', 'employer detail', 'etails from WOR', 'm WORKER CONFID', 'ONFIDENTIALITY ', 'LITY AGREEMENT ', 'MENT ?']


# T5

In [1]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")

def summarize(text, max_length=150):
  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

  generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

  return preds[0]

text = """SCHEDULE 1 TO EXHIBIT A
Background Information
Applied Materials, Inc. and its subsidiaries and affiliates, (collectively, “Applied Materials”), are subject to United States
export regulations. These regulations limit Applied Materials from disclosing certain types of technology, software and 
other technical data to citizens and residents (including permanent residents) of certain countries. Such disclosure may 
occur, for example, through any of the following means:
o making technology available by practice or application under the guidance of a person knowledgeable in the 
technology, 
o visual inspection of equipment or facilities, 
o oral exchange of information, and 
o application to situations abroad of personal knowledge or technical experience acquired in the United States.
"""
print(len(text))
summarized_text = summarize(text)
print("\n")
print(summarized_text)

c:\ProgramData\Anaconda3\envs\chatbot38\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\achyuta.sahoo\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


c:\ProgramData\Anaconda3\envs\chatbot38\lib\site-packages\transformers\models\auto\modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


794


permanent residents) of certain countries. The regulations limit Applied Materials from disclosing certain types of technology, software and other technical data to citizens and residents (including permanent residents) of certain countries. Such disclosure may occur, for example, through: o making technology available by practice or application under the guidance of a person knowledgeable in the technology; o visual inspection of equipment or facilities; o oral exchange of information. Applied Materials is subject to United States export regulations. Notably, these regulations limit Applied Materials from disclosing certain


In [2]:
text = """Nothing in this section 4 will be construed or interpreted to create or imply the existence of any 
employment agreement between Contingent Worker and Applied.
5. OWNERSHIP OF DOCUMENTS
All data, including drawings, specifications, designs and other information furnished by Applied to 
Contingent Worker in connection with his/her work assignment will remain the sole and exclusive property 
of Applied. The Contingent Worker shall return these items to Applied upon termination of his/her work 
assignment or at any time upon request by Applied.
6. INFORMATION OF FORMER EMPLOYER
Contingent Worker shall not disclose or use for Applied’s benefit any confidential or proprietary information 
of Contingent Worker’s former employer(s) or any other third parties to which Contingent Worker has a 
confidentiality obligation. The Contingent Worker must not bring onto the premises of Applied any nonpublic documents or any other property belonging to any former employer(s) or any other third parties unless 
permitted to do so in advance in writing by such party.
"""
print(len(text))
summarized_text = summarize(text)
print("\n")
print(summarized_text)

1064


in connection with his/her work assignment will remain the sole and exclusive property of Applied. Data, drawings, specifications, designs and other information furnished by Applied to Contingent Worker in connection with his/her work assignment will remain the sole and exclusive property of Applied. Contingent Worker must return these items to Applied upon termination of his/her work assignment or at any request by Applied. 7. INFORMATION OF FORMER EMPLOYER Contingent Worker shall not disclose or use for Applied’s benefit
